In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time

torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from sklearn.preprocessing import MinMaxScaler
#rotate a list by given offset, (negative offset, right to left) 
def rotateList(items, offset):
    return np.roll(items,offset)
    
def gaussiantimeseries(limit = 60, interval = 2):
    data = {}
    for t in range(0,limit,interval):
        data[t]=random.normal(loc = random.uniform(1,100), scale =1.5, size =1000)
    return data

def createsignal(limit = 60, interval = 2):
    signal1, signal2, offset = [],[],[]
    offset = random.randint(-5,5)
    data = gaussiantimeseries(limit, interval)
    for t in data:
        dist = data[t]
        signal1.append(random.choice(dist))
        signal2.append(random.choice(dist))
    return signal1, rotateList(signal2,offset), offset

def createsignals(num = 50, limit = 60, interval = 2, normalize = False):
    input_seq = []
    for i in range(num):
        sig1, sig2, offset = createsignal(limit = limit)
        train_data = np.stack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data.reshape(-1,1)).view(-1)
            input_seq.append((train_data, torch.FloatTensor([offset])))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))
            train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)
            input_seq.append((train_data_normalized, torch.FloatTensor([offset])))
    return input_seq
# print(createsignal())

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

Creating Data and training without normalization

In [5]:
train_input = createsignals(num=10000, normalize = False)
test_input = createsignals()
# training_length = int(len(input)*(.8))
# train_input = input[:training_length]
# val_input = input[training_length:] 

In [6]:
train_input[:1]

[(tensor([ 39.3136,  39.9405,  46.9535,  48.5853,  50.3278,  66.2109,  32.0394,
           15.3345,  42.3119,  63.9890,  93.0358,  79.7846,  51.9152,  65.4746,
           30.5768,  34.5674,  91.6256,  34.1958,  10.8733,  33.3607,  99.5029,
           52.3128,  13.7111,  41.0224,  86.1746,  20.6475,   7.3225,  96.3329,
           10.5937,  72.1510, 100.0675,  11.3663,  70.0137,  42.3683,  40.6009,
           45.6989,  50.9479,  50.2993,  63.3569,  29.3851,  17.3014,  44.4835,
           62.6480,  89.9707,  80.9381,  52.2071,  62.9261,  33.6998,  33.2581,
           94.8048,  35.8998,  11.3545,  33.5116,  96.7467,  51.3826,  13.3649,
           40.3740,  85.9993,  16.4067,   6.4366]), tensor([3.]))]

In [ ]:
model = LSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
epochs = 150
model.train()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    model.zero_grad()
    for seq, labels in train_input:

        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     print(f'epoch: {i:3} loss: {loss.item():10.8f}')

# print(f'epoch: {i:3} loss: {loss.item():10.10f}')

Epoch 1/150
----------
Epoch 1 Loss 8.265471697484703
Epoch 2/150
----------
Epoch 2 Loss 8.259073723232932
Epoch 3/150
----------
Epoch 3 Loss 8.256998374608903
Epoch 4/150
----------
Epoch 4 Loss 8.255725476868077
Epoch 5/150
----------
Epoch 5 Loss 8.254772655383125
Epoch 6/150
----------
Epoch 6 Loss 8.255879105000664
Epoch 7/150
----------
Epoch 7 Loss 8.25368494532602
Epoch 8/150
----------
Epoch 8 Loss 8.2524819595322
Epoch 9/150
----------
Epoch 9 Loss 8.251945942242862
Epoch 10/150
----------
Epoch 10 Loss 8.249477077814053
Epoch 11/150
----------
Epoch 11 Loss 8.250886402157104
Epoch 12/150
----------
Epoch 12 Loss 8.248389541610049
Epoch 13/150
----------
Epoch 13 Loss 8.247844660902237
Epoch 14/150
----------
Epoch 14 Loss 8.24627852370964
Epoch 15/150
----------
Epoch 15 Loss 8.242609364331674
Epoch 16/150
----------
Epoch 16 Loss 8.239161364138988
Epoch 17/150
----------
Epoch 17 Loss 8.232292557997297
Epoch 18/150
----------
Epoch 18 Loss 8.214286407992045
Epoch 19/150
-

In [ ]:
epochs = 1
model.eval()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    # model.zero_grad()
    for seq, labels in test_input:

        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                 torch.zeros(1, 1, model.hidden_layer_size))
        # # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        print(f"Offset {labels.data[0]}, Prediction {y_pred.data[0]}")
        # optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        # loss.backward()
        # optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     pr